In [1]:
from transformer_analysis import evaluate_transformer

/usr/lib/python3.7/site-packages/marshmallow/schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


In [2]:
from qiskit.transpiler import transpile
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
from qiskit.transpiler.passes.mapping import *
from qiskit import BasicAer
from qiskit.mapper import CouplingMap
from qiskit import IBMQ
from qiskit import QuantumRegister, QuantumCircuit

In [3]:
IBMQ.load_accounts()

In [4]:
backend = BasicAer.get_backend( 'qasm_simulator' )
backend_qx4 = IBMQ.get_backend( 'ibmqx2' )

pm_optimize = PassManager()
pm_optimize.append( CXCancellation()  )
pm_optimize.append( Optimize1qGates() )

pm_unroll = PassManager()
pm_unroll.append( Unroller( ['u3', 'cx'] ) )

In [5]:
couplingmap = CouplingMap([ [0, 1], [0, 2] ])

In [6]:
def simple_optimize ( circuit ):
    for i in range( 3 ):
        circuit = transpile( circuit, backend, pass_manager=pm_optimize )
    return circuit

def simple_unroll ( circuit ):
    return transpile( circuit, backend, pass_manager=pm_unroll )

In [7]:
def basic_compiler ( circuit ):
    circuit = simple_optimize( circuit )
    circuit = transpile( circuit, backend, pass_manager=PassManager( BasicSwap( couplingmap ) ) )
    circuit = simple_optimize( circuit )
    return circuit

In [8]:
def lookahead_compiler ( circuit ):
    circuit = simple_optimize( circuit )
    circuit = transpile( circuit, backend, pass_manager=PassManager( LookaheadSwap( couplingmap ) ) )
    circuit = simple_optimize( circuit )
    return circuit

In [9]:
def stochastic_compiler ( circuit ):
    circuit = simple_optimize( circuit )
    circuit = transpile( circuit, backend, pass_manager=PassManager( StochasticSwap( couplingmap ) ) )
    circuit = simple_optimize( circuit )
    return circuit

In [10]:
qreg = QuantumRegister( 3, 'q' )
circ = QuantumCircuit( qreg )
circ.ccx( qreg[0], qreg[1], qreg[2] )

In [11]:
evaluate_transformer( stochastic_compiler )

0.5759259307739605

In [12]:
evaluate_transformer( lookahead_compiler )

0.23041557580111918

In [13]:
evaluate_transformer( basic_compiler )

0.43593727650212977